# Actividad 1 - Módulo 3

### Configuración inicial

In [ ]:
# Carga de librerías
import numpy as np

### 1. Generar arreglo 3D con datos de sensores

In [ ]:
# Generar datos para cada variable
temperatura = np.random.uniform(5, 110, (30, 96))
presion = np.random.uniform(80, 140, (30, 96))
humedad = np.random.uniform(15, 85, (30, 96))

# Concatenar en la tercera dimensión
D = np.stack([temperatura, presion, humedad], axis=2)

### 2. Introducir valores perdidos y atípicos

In [ ]:
# Hacer una copia para preservar los datos originales
D_original = D.copy()

# Introducir 30 valores perdidos (np.nan)
total_elementos = D.size
indices_nan = np.random.choice(total_elementos, 30, replace=False)
D_flat = D.flatten()
D_flat[indices_nan] = np.nan
D = D_flat.reshape(D.shape)

# Introducir valores extremos (3 mayores y 3 menores por canal)
# Temperatura (canal 0)
temp_indices = np.random.choice(D.shape[0] * D.shape[1], 6, replace=False)
temp_rows = temp_indices // D.shape[1]
temp_cols = temp_indices % D.shape[1]
D[temp_rows[:3], temp_cols[:3], 0] = np.random.uniform(115, 130, 3)  # Valores mayores
D[temp_rows[3:], temp_cols[3:], 0] = np.random.uniform(-10, 4, 3)    # Valores menores

# Presión (canal 1)
pres_indices = np.random.choice(D.shape[0] * D.shape[1], 6, replace=False)
pres_rows = pres_indices // D.shape[1]
pres_cols = pres_indices % D.shape[1]
D[pres_rows[:3], pres_cols[:3], 1] = np.random.uniform(145, 160, 3)  # Valores mayores
D[pres_rows[3:], pres_cols[3:], 1] = np.random.uniform(60, 79, 3)    # Valores menores

# Humedad (canal 2)
hum_indices = np.random.choice(D.shape[0] * D.shape[1], 6, replace=False)
hum_rows = hum_indices // D.shape[1]
hum_cols = hum_indices % D.shape[1]
D[hum_rows[:3], hum_cols[:3], 2] = np.random.uniform(90, 100, 3)     # Valores mayores
D[hum_rows[3:], hum_cols[3:], 2] = np.random.uniform(0, 14, 3)       # Valores menores

### 3. Limpieza inicial

In [ ]:
# Contar valores perdidos por variable
nan_temp = np.isnan(D[:, :, 0]).sum()
nan_pres = np.isnan(D[:, :, 1]).sum()
nan_hum = np.isnan(D[:, :, 2]).sum()

print(f"Valores perdidos por variable:")
print(f"- Temperatura: {nan_temp}")
print(f"- Presión: {nan_pres}")
print(f"- Humedad: {nan_hum}")
print(f"- Total: {nan_temp + nan_pres + nan_hum}")

# Sustituir valores perdidos por la media de cada sensor
D_limpio = D.copy()
for sensor in range(D.shape[0]):
    for var in range(D.shape[2]):
        sensor_data = D[sensor, :, var]
        if np.any(np.isnan(sensor_data)):
            media_sensor = np.nanmean(sensor_data)
            mask_nan = np.isnan(sensor_data)
            D_limpio[sensor, mask_nan, var] = media_sensor

Valores perdidos por variable:
- Temperatura: 9
- Presión: 10
- Humedad: 11
- Total: 30


### 4. Filtrado y análisis condicional

In [ ]:
# Calcular temperatura máxima y humedad mínima por sensor
temp_max_por_sensor = np.max(D_limpio[:, :, 0], axis=1)
hum_min_por_sensor = np.min(D_limpio[:, :, 2], axis=1)

# Identificar sensores que cumplen ambas condiciones
sensores_filtrados = (temp_max_por_sensor > 100) & (hum_min_por_sensor < 20)
indices_sensores_filtrados = np.where(sensores_filtrados)[0]

print(f"Condiciones:")
print(f"- Temperatura máxima > 100°C")
print(f"- Humedad mínima < 20%")
print(f"\nSensores que cumplen ambas condiciones: {len(indices_sensores_filtrados)}")
print(f"Índices de sensores: {indices_sensores_filtrados}")

Condiciones:
- Temperatura máxima > 100°C
- Humedad mínima < 20%

Sensores que cumplen ambas condiciones: 30
Índices de sensores: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


### 5. Normalización y reducción

In [ ]:
# Normalizar presión [0, 1] por sensor
D_normalizado = D_limpio.copy()
for sensor in range(D.shape[0]):
    presion_sensor = D_limpio[sensor, :, 1]
    min_pres = np.min(presion_sensor)
    max_pres = np.max(presion_sensor)
    if max_pres > min_pres:
        D_normalizado[sensor, :, 1] = (presion_sensor - min_pres) / (max_pres - min_pres)

# Calcular estadísticas para sensores filtrados
if len(indices_sensores_filtrados) > 0:
    datos_filtrados = D_limpio[indices_sensores_filtrados, :, :]
    
    # Estadísticas por variable
    estadisticas = {}
    nombres_vars = ['Temperatura', 'Presión', 'Humedad']
    
    for i, var_name in enumerate(nombres_vars):
        var_data = datos_filtrados[:, :, i].flatten()
        estadisticas[var_name] = {
            'media': np.mean(var_data),
            'mediana': np.median(var_data),
            'desv_std': np.std(var_data)
        }
    
    print(f"Estadísticas de los {len(indices_sensores_filtrados)} sensores filtrados:")
    for var_name, stats in estadisticas.items():
        print(f"\n{var_name}:")
        print(f"- Media: {stats['media']:.2f}")
        print(f"- Mediana: {stats['mediana']:.2f}")
        print(f"- Desv. estándar: {stats['desv_std']:.2f}")

Estadísticas de los 30 sensores filtrados:

Temperatura:
- Media: 57.59
- Mediana: 58.39
- Desv. estándar: 30.42

Presión:
- Media: 109.93
- Mediana: 109.94
- Desv. estándar: 17.38

Humedad:
- Media: 49.51
- Mediana: 49.67
- Desv. estándar: 20.22


### 6. Correlación y análisis avanzado

In [ ]:
if len(indices_sensores_filtrados) > 0:
    # Reorganizar datos para calcular correlación
    temp_filtrada = datos_filtrados[:, :, 0].flatten()
    pres_filtrada = datos_filtrados[:, :, 1].flatten()
    hum_filtrada = datos_filtrados[:, :, 2].flatten()
    
    # Crear matriz de datos
    datos_correlacion = np.column_stack([temp_filtrada, pres_filtrada, hum_filtrada])
    
    # Calcular matriz de correlación
    matriz_corr = np.corrcoef(datos_correlacion.T)
    print("Matriz de correlación:")
    print(matriz_corr)
    
    # Identificar variable más correlacionada con temperatura
    corr_con_temp = np.abs(matriz_corr[0, 1:])
    var_mas_correlacionada = ['Presión', 'Humedad'][np.argmax(corr_con_temp)]
    print(f"\nVariable más correlacionada con temperatura: {var_mas_correlacionada}")
    print(f"Coeficiente de correlación: {matriz_corr[0, np.argmax(corr_con_temp) + 1]:.4f}")

Matriz de correlación:
[[ 1.          0.01530581 -0.00227898]
 [ 0.01530581  1.         -0.00154111]
 [-0.00227898 -0.00154111  1.        ]]

Variable más correlacionada con temperatura: Presión
Coeficiente de correlación: 0.0153


### 7. Álgebra lineal aplicada

In [ ]:
# Calcular varianza de temperatura para cada sensor
varianzas_temp = np.var(D_limpio[:, :, 0], axis=1)
sensor_max_var = np.argmax(varianzas_temp)

print(f"Sensor con mayor varianza en temperatura: Sensor {sensor_max_var}")
print(f"Varianza: {varianzas_temp[sensor_max_var]:.2f}")

# Datos del sensor con mayor varianza
datos_sensor = D_limpio[sensor_max_var, :, :].T  # Transponer para tener variables en filas

# Calcular matriz de covarianza
matriz_cov = np.cov(datos_sensor)
print("Matriz de covarianza:")
print(matriz_cov)

# Descomposición en valores y vectores propios
valores_propios, vectores_propios = np.linalg.eig(matriz_cov)

# Ordenar por valores propios descendentes
indices_orden = np.argsort(valores_propios)[::-1]
valores_propios = valores_propios[indices_orden]
vectores_propios = vectores_propios[:, indices_orden]

print("\nValores propios (ordenados):")
for i, val in enumerate(valores_propios):
    porcentaje = (val / np.sum(valores_propios)) * 100
    print(f"- λ{i+1} = {val:.2f} ({porcentaje:.1f}% de la varianza)")

print("\nVectores propios correspondientes:")
for i in range(3):
    print(f"- v{i+1} = [{vectores_propios[0,i]:.3f}, {vectores_propios[1,i]:.3f}, {vectores_propios[2,i]:.3f}]")

# Interpretación
print("\nInterpretación:")
componente_principal = np.abs(vectores_propios[:, 0])
var_principal = ['Temperatura', 'Presión', 'Humedad'][np.argmax(componente_principal)]
print(f"La variable que más aporta a la variabilidad es: {var_principal}")
print(f"Con una contribución de {np.max(componente_principal):.3f} en el primer componente principal")
print(f"El primer componente explica el {(valores_propios[0]/np.sum(valores_propios))*100:.1f}% de la varianza total")

Sensor con mayor varianza en temperatura: Sensor 9
Varianza: 1096.70
Matriz de covarianza:
[[1108.24706937 -120.43737697  107.1148067 ]
 [-120.43737697  258.48927141  -51.22518853]
 [ 107.1148067   -51.22518853  465.80516274]]

Valores propios (ordenados):
- λ1 = 1143.91 (62.4% de la varianza)
- λ2 = 452.89 (24.7% de la varianza)
- λ3 = 235.74 (12.9% de la varianza)

Vectores propios correspondientes:
- v1 = [0.976, -0.142, 0.165]
- v2 = [-0.185, -0.142, 0.972]
- v3 = [0.115, 0.980, 0.165]

Interpretación:
La variable que más aporta a la variabilidad es: Temperatura
Con una contribución de 0.976 en el primer componente principal
El primer componente explica el 62.4% de la varianza total
